In [1]:
# bibliotecas

import zipfile
import boto3
from io import BytesIO

In [46]:
# variáveis de entrada
nome_bucket = 'elasticbeanstalk-us-east-2-047710767346'
nome_subpasta = 'Case_A3'
pasta_origem = 'arquivos_zip'
pasta_destino = 'arquivos_extraidos'

s3 = boto3.client('s3')

In [49]:
def extrair_arquivos_zip(nome_bucket, nome_subpasta, pasta_origem, pasta_destino):
    # verificar se a pasta destino existe e, senão, criá-la
    try:
        s3.head_object(Bucket=nome_bucket, Key=f'{nome_subpasta}/{pasta_destino}/')
    except:
        s3.put_object(Bucket=nome_bucket, Key=f'{nome_subpasta}/{pasta_destino}/')

    # listar os arquivos zip na pasta "arquivos_zip"
    objetos = s3.list_objects(Bucket=nome_bucket, Prefix=f'{nome_subpasta}/{pasta_origem}')

    # iterar sobre os arquivos zip e extraí-los
    for obj in objetos.get('Contents', []):
        nome_zip = obj['Key'].split('/')[-1]

        try:
            # verificar se o arquivo é um ZIP
            if not nome_zip.endswith('.zip'):
                raise ValueError(f'O arquivo {nome_zip} não é um arquivo ZIP válido.')

            # obter o conteúdo do arquivo zip
            response = s3.get_object(Bucket=nome_bucket, Key=obj['Key'])
            conteudo_zip = response['Body'].read()

            # criar uma subpasta única para cada arquivo dentro do S3
            subpasta_arquivo = f'{nome_subpasta}/{pasta_destino}/{nome_zip.replace(".zip", "")}/'
            s3.put_object(Bucket=nome_bucket, Key=subpasta_arquivo)

            # extrair o arquivo zip para a subpasta específica dentro do S3
            with zipfile.ZipFile(BytesIO(conteudo_zip)) as zip_ref:
                for file_info in zip_ref.infolist():
                    s3.put_object(
                        Bucket=nome_bucket,
                        Key=subpasta_arquivo + file_info.filename,
                        Body=zip_ref.read(file_info.filename)
                    )

            # imprimir mensagem de confirmação personalizada
            ano = nome_zip.split("_")[0] if nome_zip.split("_")[0].isdigit() else "desconhecido"
            print(f'O arquivo com os dados de {ano} foi extraído para a pasta {subpasta_arquivo}')
        except Exception as e:
            print(f'Erro ao extrair o arquivo {nome_zip}: {e}')

# extraindo os arquivos zip
extrair_arquivos_zip(nome_bucket, nome_subpasta, pasta_origem, pasta_destino)

Erro ao extrair o arquivo : O arquivo  não é um arquivo ZIP válido.
O arquivo com os dados de 1995 foi extraído para a pasta Case_A3/arquivos_extraidos/1995_microdados_censo_da_educacao_superior_1995/
O arquivo com os dados de 1996 foi extraído para a pasta Case_A3/arquivos_extraidos/1996_microdados_censo_da_educacao_superior_1996/
O arquivo com os dados de 1997 foi extraído para a pasta Case_A3/arquivos_extraidos/1997_microdados_censo_da_educacao_superior_1997/
O arquivo com os dados de 1998 foi extraído para a pasta Case_A3/arquivos_extraidos/1998_microdados_censo_da_educacao_superior_1998/
O arquivo com os dados de 1999 foi extraído para a pasta Case_A3/arquivos_extraidos/1999_microdados_censo_da_educacao_superior_1999/
O arquivo com os dados de 2000 foi extraído para a pasta Case_A3/arquivos_extraidos/2000_microdados_censo_da_educacao_superior_2000/
O arquivo com os dados de 2001 foi extraído para a pasta Case_A3/arquivos_extraidos/2001_microdados_censo_da_educacao_superior_2001/
O